# [SOLUTION] Exercise - Building a Multi-Step State Machine Agent

In this exercise, you will build an agent that manages a multi-step workflow using a state machine. You’ll define a custom state schema, implement step logic, connect steps (including conditional routing and loops), and run the workflow to process user input through several transformations.

## Challenge

You have learned how to use a state machine to manage workflow steps and transitions. Now, your challenge is to:

- Define a state schema with multiple fields (e.g., user_query, instructions, messages, current_tool_calls).
- Implement at least three step functions:
    - Prepare Messages: Assemble the conversation history and any required context for the LLM.
    - LLM: Call the language model to generate a response or tool call.
    - Tools: Execute any required tool calls and update the state with results.
- Connect steps to form a workflow, including:
    - Entrypoint and Termination steps to start and end the workflow.
    - Conditional routing: If the LLM response includes tool calls, route to the Tools step; otherwise, proceed to Termination.
    - Looping: After executing tools, return to the LLM step to continue the workflow until there are no more tool calls.
- Run your state machine with a sample input and inspect the state transitions and snapshots to understand how your agent processes a task step by step.


## Setup
First, let's import the necessary libraries:

In [1]:
from typing import TypedDict, List, Optional, Union
import json
from dotenv import load_dotenv
import os

from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Run
from lib.llm import LLM
from lib.messages import AIMessage, UserMessage, SystemMessage, ToolMessage
from lib.tooling import Tool, ToolCall, tool

In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_key

'voc-2015200036168865441126268e0ed5be47456.24871461'

## Define a State Schema

Create a TypedDict to represent the agent’s state, including fields for the user query, instructions, message history, and any pending tool calls.

In [3]:
class AgentState(TypedDict):
    user_query: str  # The current user query being processed
    instructions: str  # System instructions for the agent
    messages: List[dict]  # List of conversation messages
    current_tool_calls: Optional[List[ToolCall]]  # Current pending tool calls

## Define the Tools you will use

Feel free to modify to add any tool you want

In [4]:
@tool
def get_games(num_games:int=1, top:bool=True) -> str:
    """
    Returns the top or bottom N games with highest or lowest scores.    
    args:
        num_games (int): Number of games to return (default is 1)
        top (bool): If True, return top games, otherwise return bottom (default is True)
    """
    data = [
        {"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98},
        {"Game": "Super Mario Odyssey", "Platform": "Switch", "Score": 97},
        {"Game": "Metroid Prime", "Platform": "GameCube", "Score": 97},
        {"Game": "Super Smash Bros. Brawl", "Platform": "Wii", "Score": 93},
        {"Game": "Mario Kart 8 Deluxe", "Platform": "Switch", "Score": 92},
        {"Game": "Fire Emblem: Awakening", "Platform": "3DS", "Score": 92},
        {"Game": "Donkey Kong Country Returns", "Platform": "Wii", "Score": 87},
        {"Game": "Luigi's Mansion 3", "Platform": "Switch", "Score": 86},
        {"Game": "Pikmin 3", "Platform": "Wii U", "Score": 85},
        {"Game": "Animal Crossing: New Leaf", "Platform": "3DS", "Score": 88}
    ]
    # Sort the games list by Score
    # If top is True, descending order
    sorted_games = sorted(data, key=lambda x: x['Score'], reverse=top)
    
    # Return the N games
    return sorted_games[:num_games]

In [5]:
tools = [get_games]

## Create the Steps

Write functions for each step in your workflow:


**Prepare Messages**: Build the message list for the LLM.

In [6]:
def prepare_messages_step(state: AgentState) -> AgentState:
    """Step logic: Prepare messages for LLM consumption"""

    messages = [
        SystemMessage(content=state["instructions"]),
        UserMessage(content=state["user_query"])
    ]
    
    return {
        "messages": messages
    }

**LLM Step**: Call the language model and check for tool calls.

In [7]:
def llm_step(state: AgentState) -> AgentState:
    """Step logic: Process the current state through the LLM"""

    # Initialize LLM
    llm = LLM(
        model="gpt-4o-mini",
        temperature=0.3,
        tools=tools,
        api_key=openai_api_key,
    )

    response = llm.invoke(state["messages"])
    tool_calls = response.tool_calls if response.tool_calls else None

    # Create AI message with content and tool calls
    ai_message = AIMessage(content=response.content, tool_calls=tool_calls)
    
    return {
        "messages": state["messages"] + [ai_message],
        "current_tool_calls": tool_calls
    }

**Tool Step**: Execute any tool calls and update the state.

In [8]:
def tool_step(state: AgentState) -> AgentState:
    """Step logic: Execute any pending tool calls"""
    tool_calls = state["current_tool_calls"] or []
    tool_messages = []
    
    for call in tool_calls:
        # Access tool call data correctly
        function_name = call.function.name
        function_args = json.loads(call.function.arguments)
        tool_call_id = call.id
        # Find the matching tool
        tool = next((t for t in tools if t.name == function_name), None)
        if tool:
            result = tool(**function_args)
            tool_messages.append(
                ToolMessage(
                    content=json.dumps(result), 
                    tool_call_id=tool_call_id, 
                    name=function_name, 
                )
            )
    
    # Clear tool calls and add results to messages
    return {
        "messages": state["messages"] + tool_messages,
        "current_tool_calls": None
    }

## Build and Connect the State Machine

Add your steps to the state machine, and connect them with transitions. Use conditional routing to decide whether to call tools or terminate, and loop as needed.

In [9]:
workflow = StateMachine[AgentState](AgentState)

In [10]:
# Create steps
entry = EntryPoint[AgentState]()
message_prep = Step[AgentState]("message_prep", prepare_messages_step)
llm_processor = Step[AgentState]("llm_processor", llm_step)
tool_executor = Step[AgentState]("tool_executor", tool_step)
termination = Termination[AgentState]()
        
workflow.add_steps(
    [
        entry, 
        message_prep, 
        llm_processor, 
        tool_executor, 
        termination
    ]
)

In [11]:
# Add transitions
workflow.connect(entry, message_prep)
workflow.connect(message_prep, llm_processor)

# Transition based on whether there are tool calls
def check_tool_calls(state: AgentState) -> Union[Step[AgentState], str]:
    """Transition logic: Check if there are tool calls"""
    if state.get("current_tool_calls"):
        return tool_executor
    return termination

# Routing: If tool calls -> tool_executor
workflow.connect(
    source=llm_processor, 
    targets=[tool_executor, termination], 
    condition=check_tool_calls
)

# Looping: Go back to llm after tool execution
workflow.connect(
    source=tool_executor, 
    targets=llm_processor
)

## Run the Workflow

In [12]:
initial_state: AgentState = {
    "user_query": "What's the best game in the dataset?",
    "instructions": "You can bring insights about a game dataset based on users questions",
    "messages": [],
}

In [13]:
run_object = workflow.run(initial_state)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
invoke
else in invoke


APIConnectionError: Connection error.

In [ ]:
run_object.get_final_state()["messages"]

[SystemMessage(role='system', content='You can bring insights about a game dataset based on users questions'),
 UserMessage(role='user', content="What's the best game in the dataset?"),
 AIMessage(role='assistant', content=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yxCOOsxBkcbjOhJIit1Akdhw', function=Function(arguments='{"num_games":1,"top":true}', name='get_games'), type='function')]),
 ToolMessage(role='tool', content='[{"Game": "The Legend of Zelda: Breath of the Wild", "Platform": "Switch", "Score": 98}]', tool_call_id='call_yxCOOsxBkcbjOhJIit1Akdhw', name='get_games'),
 AIMessage(role='assistant', content='The best game in the dataset is **The Legend of Zelda: Breath of the Wild** for the Switch, with a score of **98**.', tool_calls=None)]

## Optional 

Create an Agent class to encapsulate State Machine logic. Then try adding more tools, and experiment with different user queries to see how the workflow adapts.

In [ ]:
class Agent:
    def __init__(self, 
                 model_name: str,
                 instructions: str, 
                 tools: List[Tool] = None,
                 temperature: float = 0.7):
        """
        Initialize an Agent instance
        
        Args:
            model_name: Name/identifier of the LLM model to use
            instructions: System instructions for the agent
            tools: Optional list of tools available to the agent
            temperature: Temperature parameter for LLM (default: 0.7)
        """
        self.instructions = instructions
        self.tools = tools if tools else []
        self.model_name = model_name
        self.temperature = temperature
                
        # Initialize state machine
        self.workflow = self._create_state_machine()

    def _prepare_messages_step(self, state: AgentState) -> AgentState:
        """Step logic: Prepare messages for LLM consumption"""

        messages = [
            SystemMessage(content=state["instructions"]),
            UserMessage(content=state["user_query"])
        ]
        
        return {
            "messages": messages
        }

    def _llm_step(self, state: AgentState) -> AgentState:
        """Step logic: Process the current state through the LLM"""

        # Initialize LLM
        llm = LLM(
            model=self.model_name,
            temperature=self.temperature,
            tools=self.tools
        )

        response = llm.invoke(state["messages"])
        tool_calls = response.tool_calls if response.tool_calls else None

        # Create AI message with content and tool calls
        ai_message = AIMessage(content=response.content, tool_calls=tool_calls)
        
        return {
            "messages": state["messages"] + [ai_message],
            "current_tool_calls": tool_calls
        }

    def _tool_step(self, state: AgentState) -> AgentState:
        """Step logic: Execute any pending tool calls"""
        tool_calls = state["current_tool_calls"] or []
        tool_messages = []
        
        for call in tool_calls:
            # Access tool call data correctly
            function_name = call.function.name
            function_args = json.loads(call.function.arguments)
            tool_call_id = call.id
            # Find the matching tool
            tool = next((t for t in self.tools if t.name == function_name), None)
            if tool:
                result = tool(**function_args)
                tool_messages.append(
                    ToolMessage(
                        content=json.dumps(result), 
                        tool_call_id=tool_call_id, 
                        name=function_name, 
                    )
                )
        
        # Clear tool calls and add results to messages
        return {
            "messages": state["messages"] + tool_messages,
            "current_tool_calls": None
        }

    def _create_state_machine(self) -> StateMachine[AgentState]:
        """Create the internal state machine for the agent"""
        machine = StateMachine[AgentState](AgentState)
        
        # Create steps
        entry = EntryPoint[AgentState]()
        message_prep = Step[AgentState]("message_prep", self._prepare_messages_step)
        llm_processor = Step[AgentState]("llm_processor", self._llm_step)
        tool_executor = Step[AgentState]("tool_executor", self._tool_step)
        termination = Termination[AgentState]()
        
        machine.add_steps([entry, message_prep, llm_processor, tool_executor, termination])
        
        # Add transitions
        machine.connect(entry, message_prep)
        machine.connect(message_prep, llm_processor)
        
        # Transition based on whether there are tool calls
        def check_tool_calls(state: AgentState) -> Union[Step[AgentState], str]:
            """Transition logic: Check if there are tool calls"""
            if state.get("current_tool_calls"):
                return tool_executor
            return termination
        
        machine.connect(llm_processor, [tool_executor, termination], check_tool_calls)
        machine.connect(tool_executor, llm_processor)  # Go back to llm after tool execution
        
        return machine

    def invoke(self, query: str) -> Run:
        """
        Run the agent on a query
        
        Args:
            query: The user's query to process
            
        Returns:
            The final run object after processing
        """

        initial_state: AgentState = {
            "user_query": query,
            "instructions": self.instructions,
            "messages": [],
        }

        run_object = self.workflow.run(initial_state)

        return run_object


In [ ]:
@tool
def power(base:float, exponent:float):
    """Exponentatiation: base to the power of exponent"""
    
    return base ** exponent

In [ ]:
@tool
def multiply(number_a:float, number_b:float):
    """Multiplication: number_a times number_b"""
    
    return number_a * number_b

In [ ]:
tools = [power, multiply]

In [ ]:
math_agent = Agent(
    model_name="gpt-4o-mini",
    tools=tools,
    instructions=(
        "You're an AI Agent very good with math operations "
        "You can answer multistep questions by sequentially calling functions. "
        "You follow a pattern of of Thought and Action. "
        "Create a plan of execution: "
        "- Use Thought to describe your thoughts about the question you have been asked. "
        "- Use Action to specify one of the tools available to you. if you don't have a tool available, you can respond directly."
        "When you think it's over, return the answer "
        "Never try to respond directly if the question needs a tool. "
        "But if you don't have a tool available, you can respond directly. "
        f"The actions you have are the Tools: {tools}. \n"
    )
)

In [ ]:
run_object = math_agent.invoke(
    query="What's 3 to the power of 2? Take the result, then multiply it by 5.",
)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [ ]:
run_object.get_final_state()["messages"]

[SystemMessage(role='system', content="You're an AI Agent very good with math operations You can answer multistep questions by sequentially calling functions. You follow a pattern of of Thought and Action. Create a plan of execution: - Use Thought to describe your thoughts about the question you have been asked. - Use Action to specify one of the tools available to you. if you don't have a tool available, you can respond directly.When you think it's over, return the answer Never try to respond directly if the question needs a tool. But if you don't have a tool available, you can respond directly. The actions you have are the Tools: [<Tool name=power params=['base', 'exponent']>, <Tool name=multiply params=['number_a', 'number_b']>]. \n"),
 UserMessage(role='user', content="What's 3 to the power of 2? Take the result, then multiply it by 5."),
 AIMessage(role='assistant', content='Thought: First, I need to calculate \\(3\\) to the power of \\(2\\). Then, I will take that result and mult